In [4]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Workspace

ws = Workspace.from_config()

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config
script_config = ScriptRunConfig(source_directory='training_folder',
                                script='training_script.py',
                                arguments = ['--reg-rate', 0.1],
                                environment=sklearn_env)
# Submit the experiment
experiment = Experiment(workspace=ws, name='training-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion()

{'runId': 'training-experiment_1610127238_9742346f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-01-08T17:34:02.119252Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'ad4c6059-405d-4c8e-a994-160f22329a6a',
  'azureml.git.repository_uri': 'https://github.com/Cloblak/Azure_MLCert_WorkingDir.git',
  'mlflow.source.git.repoURL': 'https://github.com/Cloblak/Azure_MLCert_WorkingDir.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '0971652cbaa72af7a20789ea538d0f946e3ec6f1',
  'mlflow.source.git.commit': '0971652cbaa72af7a20789ea538d0f946e3ec6f1',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'training_script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg-rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  

In [5]:
# "run" is a reference to a completed experiment run

# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

# Download a named file
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/7792_azureml.log
outputs/model.pkl


In [7]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model',
                       tags={'data-format': 'CSV', 'Accuracy': "0.77"},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model classification_model
